In [1]:
import pandas as pd
import pickle
import matplotlib.pyplot as plt
import numpy as np
import re

In [2]:
with open('all_cards', 'rb') as f:
    all_cards = pickle.load(f)

In [3]:
pd.set_option('max_columns', None)
all_cards.sample(1)

,name,released_at,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,legalities,games,reserved,foil,nonfoil,finishes,reprint,variation,set,set_name,set_type,rarity,flavor_text,artist,edhrec_rank,penny_rank,prices,produced_mana,loyalty,card_faces,life_modifier,hand_modifier,Normal_EDH_Rank,Binary_Rank
49839,spirit,2018-06-08,,0.0,token creature — spirit,flying,1,1,[W],[W],[Flying],"{'standard': 'not_legal', 'future': 'not_legal...",[paper],False,False,True,[nonfoil],True,False,tbbd,battlebond tokens,token,common,NaN,ryan yee,NaN,NaN,"{'usd': '0.06', 'usd_foil': None, 'usd_etched'...",NaN,NaN,NaN,NaN,NaN,NaN,Below_Avg


In [4]:
all_cards['double_sided'] = all_cards['card_faces'].notnull().astype("bool")

In [5]:
all_cards['planeswalker'] = all_cards['loyalty'].notnull().astype("bool")

In [6]:
all_cards['makes_mana'] = all_cards['produced_mana'].notnull().astype("bool")

In [7]:
all_cards['hand_modify'] = all_cards['hand_modifier']
all_cards['hand_modifier'] = all_cards['hand_modifier'].notnull().astype("bool")

In [8]:
all_cards['life_modify'] = all_cards['life_modifier']
all_cards['life_modifier'] = all_cards['life_modifier'].notnull().astype("bool")

In [9]:
all_cards['type_line'] = all_cards['type_line'].fillna('False')
all_cards['lands'] = all_cards['type_line'].apply(lambda x: True if 'land' in x else False)
all_cards['creature'] = all_cards['type_line'].apply(lambda x: True if 'creature' in x else False)
all_cards['enchantment'] = all_cards['type_line'].apply(lambda x: True if 'enchantment' in x else False)
all_cards['artifact'] = all_cards['type_line'].apply(lambda x: True if 'artifact' in x else False)
all_cards['instant'] = all_cards['type_line'].apply(lambda x: True if 'instant' in x else False)
all_cards['sorcery'] = all_cards['type_line'].apply(lambda x: True if 'sorcery' in x else False)

In [10]:
all_cards['legendary'] = all_cards['type_line'].apply(lambda x: True if 'legendary' in x else False)

In [11]:
all_cards['oracle_text'] = all_cards['oracle_text'].fillna('')
all_cards['cmc'] = all_cards['cmc'].fillna(0.0)

In [12]:
all_cards['prices'] = all_cards['prices'].apply(lambda x: x.get('usd'))
all_cards['prices'] = all_cards['prices'].fillna(0.0)
all_cards['prices'] = all_cards['prices'].apply(lambda x: float(x))

In [13]:
rare_dummies = pd.get_dummies(all_cards['rarity'],drop_first=True, columns = 'Binary_Rank')
all_cards = all_cards.join(rare_dummies)

In [14]:
set_type_dummies = pd.get_dummies(all_cards['set_type'],drop_first=True, columns = 'Binary_Rank')
all_cards = all_cards.join(set_type_dummies)

In [15]:
all_cards['color_identity'] = all_cards['color_identity'].apply(lambda x: ['Colorless'] if x == [] else x)
all_cards['color_identity'] = all_cards['color_identity'].apply(tuple)
color_dummies = pd.get_dummies(all_cards['color_identity'],drop_first=True, columns = 'Binary_Rank')
all_cards = all_cards.join(color_dummies)

In [16]:
split_char = '—'

def tribe_splitter(x):
    tribes = x.split(split_char, 1)
    return tribes[1]

In [17]:
all_cards['Tribes'] = all_cards['type_line'].apply(lambda x: tribe_splitter(x) if split_char in x else 'no_tribe')

In [18]:
# all_cards['Tribes']

In [19]:
tribe_set = set(['no_tribe'])

def tribe_collecting(x):
    tribe_list = x.split()
    for tribe in tribe_list:
        tribe_set.add(tribe)

In [20]:
all_cards['Tribes'].apply(lambda x: tribe_collecting(x))

droptribes = ['—', 'the', 'sorcery', 'shadowmoor', "serra’s", 'realm', 'rath', 'rabiah', 'of', 'new', 'nastiest,', 'mercadia',
              'ir', 'iquatana', 'instant', 'innistrad', 'ergamon', 'equilor', 'elemental?', 'dominaria', "bolas’s", 'biggest,',
              'baddest,', 'and/or', '//', 'zendikar', 'xerex', 'vryn', 'valla', 'ulgrotha', 'shandalar', 'segovia', 'regatha',
              'ravnica', 'pyrulea', 'proper', 'phyrexia', 'muraganda', 'mongseng', 'moag', 'mirrodin', 'meditation', 'luvion',
              'lorwyn', 'legendary', 'legend', 'kyneth', 'kolbahan', 'kinshala', 'kephalai', 'karsus', 'kamigawa', 'kaldheim',
              'etiquette', 'card', 'belenon', 'azgol', 'arkhos', 'alara', 'planeswalker', 'creature']

for nontribe in droptribes:
    tribe_set.remove(nontribe)

In [21]:
tribe_set

{'abian',
 'adventure',
 'advisor',
 'aetherborn',
 'ajani',
 'alicorn',
 'alien',
 'ally',
 'aminatou',
 'angel',
 'angrath',
 'antelope',
 'ape',
 'arcane',
 'archer',
 'archon',
 'arlinn',
 'army',
 'art',
 'artifact',
 'artificer',
 'ashiok',
 'assassin',
 'assembly-worker',
 'astartes',
 'atog',
 'aura',
 'aurochs',
 'autobot',
 'avatar',
 'azra',
 'b.o.b.',
 'background',
 'badger',
 'bahamut',
 'barbarian',
 'bard',
 'basilisk',
 'basri',
 'bat',
 'bear',
 'beast',
 'beaver',
 'beeble',
 'beholder',
 'berserker',
 'bird',
 'blood',
 'boar',
 'bolas',
 'brainiac',
 'bringer',
 'brushwagg',
 'bureaucrat',
 'calix',
 'camel',
 'carrier',
 'cartouche',
 'cat',
 'centaur',
 'cephalid',
 'chameleon',
 'chandra',
 'chicken',
 'child',
 'chimera',
 'citizen',
 'clamfolk',
 'class',
 'cleric',
 'cloud',
 'clown',
 'clue',
 'cockatrice',
 'construct',
 'contraption',
 'cow',
 'coward',
 'crab',
 'crocodile',
 'curse',
 'cyborg',
 'cyclops',
 'dack',
 'dakkon',
 'daretti',
 'dauthi',
 'dav

In [22]:
# dropplings = ['released_at', 'cmc', 'power', 'toughness', 'colors', 'color_identity', 'keywords', 'legalities', 'games', 
#               'reserved', 'foil', 'nonfoil', 'finishes', 'reprint', 'variation', 'set', 'flavor_text', 'artist', 'penny_rank', 
#               'prices', 'produced_mana', 'loyalty', 'card_faces', 'life_modifier', 'hand_modifier', 'Normal_EDH_Rank', 
#               'Binary_Rank', 'double_sided', 'planeswalker', 'makes_mana', 'hand_modify', 'life_modify', 'lands', 'creature', 
#               'enchantment', 'artifact', 'instant', 'sorcery', 'common', 'mythic', 'rare', 'special', 'uncommon', 'archenemy', 
#               'arsenal', 'box', 'commander', 'core', 'draft_innovation', 'duel_deck', 'expansion', 'from_the_vault', 'funny', 
#               'masterpiece', 'masters', 'memorabilia', 'planechase', 'premium_deck', 'promo', 'spellbook', 'starter', 'token', 
#               'treasure_chest', 'vanguard', 'Tribes', 'legendary']

# short_df = all_cards.drop(dropplings, axis = 1)

In [23]:
# short_df[short_df['type_line'].str.contains('ship')].shape

In [24]:
# short_df[short_df['type_line'].str.contains('orc')]

In [25]:
def tribe_matching(x):
    return x.split()

for tribe in tribe_set:
    all_cards[tribe] = all_cards['type_line'].apply(lambda x: True if tribe in tribe_matching(x) else False)

C:\Users\mitch\AppData\Local\Temp/ipykernel_18540/1663764669.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  all_cards[tribe] = all_cards['type_line'].apply(lambda x: True if tribe in tribe_matching(x) else False)


In [26]:
all_cards['keywords'] = all_cards['keywords'].apply(lambda x: ['no_keywords'] if x == [] else x)

In [27]:
keyword_set = set(['no_keywords'])

def keyword_collecting(x):
    for keyword in x:
        keyword_set.add(keyword)
        
all_cards['keywords'].apply(lambda x: keyword_collecting(x))

0        None
1        None
2        None
3        None
4        None
         ... 
70877    None
70878    None
70880    None
70881    None
70882    None
Name: keywords, Length: 68669, dtype: object

In [28]:
def keyword_matching(x):
    return x.split()

for keyword in keyword_set:
    all_cards[keyword] = all_cards['keywords'].apply(lambda x: True if keyword in x else False)

C:\Users\mitch\AppData\Local\Temp/ipykernel_18540/2873641389.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  all_cards[keyword] = all_cards['keywords'].apply(lambda x: True if keyword in x else False)


In [29]:
all_cards.drop(columns = ['no_keywords', 'no_tribe'], axis = 1, inplace = True)

In [39]:
pd.set_option('max_columns', None)
all_cards.sample(1)

,cmc,reserved,foil,nonfoil,reprint,variation,prices,life_modifier,hand_modifier,Binary_Rank,double_sided,planeswalker,makes_mana,lands,creature,enchantment,artifact,instant,sorcery,legendary,common,mythic,rare,special,uncommon,archenemy,arsenal,box,commander,core,draft_innovation,duel_deck,expansion,from_the_vault,funny,masterpiece,masters,memorabilia,planechase,premium_deck,promo,spellbook,starter,token,treasure_chest,vanguard,"(B, G)","(B, G, R)","(B, G, R, U)","(B, G, R, U, W)","(B, G, R, W)","(B, G, U)","(B, G, U, W)","(B, G, W)","(B, R)","(B, R, U)","(B, R, U, W)","(B, R, W)","(B, U)","(B, U, W)","(B, W)","(Colorless,)","(G,)","(G, R)","(G, R, U)","(G, R, U, W)","(G, R, W)","(G, U)","(G, U, W)","(G, W)","(R,)","(R, U)","(R, U, W)","(R, W)","(U,)","(U, W)","(W,)",barbarian,chameleon,jackal,otter,domri,rune,mouse,dihada,leech,urza,shaman,dakkon,jace,kithkin,advisor,pangolin,dack,shrine,forest,key,moonfolk,pegasus,hero,gremlin,ogre,hellion,tezzeret,cartouche,jellyfish,harpy,monk,manticore,ooze,gith,horse,daretti,hippogriff,grist,fortification,warrior,venser,avatar,ugin,oyster,archon,townsfolk,yanggu,clown,gnome,octopus,centaur,zubera,myr,deer,robot,homarid,kraken,xenagos,ally,sheep,lair,hamster,pest,minotaur,hyena,vampire,walrus,pilot,sphinx,rebel,dungeon,teyo,bolas,brainiac,serra,spike,druid,fungus,kangaroo,sable,flagbearer,paratrooper,scientist,estrid,angel,fire,gnoll,wurm,ouphe,ox,class,lhurgoyf,siren,lammasu,killbot,shark,master,kobold,aminatou,crocodile,kasmina,merfolk,villain,masticore,mountain,homunculus,cow,mine,zariel,vedalken,cat,volver,insect,demigod,spider,egg,juggernaut,berserker,elder,thrull,army,bat,dwarf,specter,turtle,worm,lady,nomad,sorin,art,plains,saheeli,halfling,spirit,citizen,gate,camel,astartes,elspeth,head,igpay,clue,frog,mordenkainen,narset,orc,golem,metathran,fabacin,mole,spy,yeti,peasant,gold,donkey,dinosaur,whale,surrakar,kor,imp,mime,brushwagg,teddy,kavu,chandra,goblin,abian,unicorn,karn,kiora,trilobite,hatificer,incarnation,treasure,inkling,beast,duck,ral,scorpion,tamiyo,teferi,samurai,aura,chicken,naga,land,hydra,eye,squirrel,will,elemental,kirin,devil,nephilim,ape,reflection,sarkhan,sponge,assembly-worker,sculpture,chimera,monger,autobot,lobster,saga,fish,triskelavite,werewolf,rowan,guest,coward,blood,desert,thalakos,satyr,koth,ellywick,spawn,huatli,survivor,phelddagrif,snake,slug,antelope,kaya,tyvar,drake,spellshaper,horror,monkey,waiter,beeble,mystic,tibalt,lukka,child,ferret,troll,dog,vivien,gorgon,drone,atog,nixilis,thopter,lizard,processor,aurochs,alicorn,arcane,noggle,illusion,locus,szat,rat,clamfolk,gargoyle,lamia,god,phoenix,aetherborn,mummy,weird,freyalise,tiefling,knight,hippo,beaver,power-plant,ranger,vampyre,garruk,minion,oko,dauthi,pirate,scion,treefolk,artificer,shade,cockatrice,boar,germ,minsc,crab,archer,plant,penguin,mercenary,reveler,ship,wolf,cloud,starfish,viashino,tasha,dryad,gus,cleric,elk,ashiok,elf,bard,gamer,scarecrow,vraska,b.o.b.,orgg,mongoose,bureaucrat,background,carrier,nymph,sliver,serf,djinn,assassin,noble,alien,eldrazi,skeleton,lolth,equipment,lesson,urza’s,dragon,trap,fractal,island,slith,wrenn,nahiri,elves,bringer,efreet,contraption,arlinn,undercity,nightstalker,fox,inzerva,yanling,grandchild,rigger,wrestler,zombie,bahamut,salamander,scout,licid,servo,demon,bird,azra,cyborg,kaito,phyrexian,mutant,basilisk,soldier,curse,wombat,jeska,rogue,raccoon,praetor,wizard,goat,jaya,nissa,cephalid,angrath,elephant,samut,wolverine,elminster,gideon,soltari,vehicle,wraith,pentavite,saproling,calix,wildfire,badger,swamp,squid,griffin,tentacle,tower,nightmare,wall,adventure,mammoth,shard,designer,shapeshifter,beholder,construct,leviathan,bear,nautilus,ajani,serpent,davriel,hag,warlock,faerie,ninja,windgrace,giant,cyclops,snow,liliana,dreadnought,rabbit,food,niko,basri,human,rhino,dovin,Battalion,Unleash,Provoke,Tribute,Scry,Basic landcycling,Champion,Recover,Eternalize,Flash,Soulshift,Assist,Nonbasic landwalk,Conspire,Landwalk,Intensity,Strive,Channel,Ingest,Adapt,Flanking,Ni

In [31]:
# drop_cols = ['released_at', 'mana_cost', 'colors', 'legalities', 'games', 'flavor_text', 'set', 'flavor_text', 'edhrec_rank',
#              'penny_rank', 'produced_mana', 'loyalty', 'card_faces', 'Normal_EDH_Rank', 'hand_modify', 'life_modify']

In [32]:
col_names = list(all_cards.columns)
len(col_names)

783

In [33]:
all_cards['Binary_Rank'] = all_cards['Normal_EDH_Rank'].apply(lambda x: 1 if x >= 0.50 else 0)

In [34]:
all_cards = all_cards.drop(columns = ['name', 'released_at', 'mana_cost', 'type_line', 'oracle_text', 'power', 'toughness',
                                      'colors', 'color_identity', 'keywords', 'legalities', 'games', 'finishes', 'set',
                                      'set_name', 'set_type', 'rarity', 'flavor_text', 'artist', 'edhrec_rank', 'penny_rank',
                                      'produced_mana', 'loyalty', 'card_faces', 'Tribes', 'Normal_EDH_Rank', 'hand_modify',
                                      'life_modify'])

In [35]:
col_names = list(all_cards.columns)
len(col_names)

755

In [36]:
all_cards.info(verbose = True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68669 entries, 0 to 70882
Data columns (total 755 columns):
 #    Column                       Dtype  
---   ------                       -----  
 0    cmc                          float64
 1    reserved                     bool   
 2    foil                         bool   
 3    nonfoil                      bool   
 4    reprint                      bool   
 5    variation                    bool   
 6    prices                       float64
 7    life_modifier                bool   
 8    hand_modifier                bool   
 9    Binary_Rank                  int64  
 10   double_sided                 bool   
 11   planeswalker                 bool   
 12   makes_mana                   bool   
 13   lands                        bool   
 14   creature                     bool   
 15   enchantment                  bool   
 16   artifact                     bool   
 17   instant                      bool   
 18   sorcery                 

In [37]:
# bin small tribes, drop those tribe columns
# convert "released_at" to just "year_released" and convert to int

# apply these additional columns to the cleaned_data notebook

In [38]:
with open('big_cards', 'wb') as f:
    pickle.dump(all_cards,f)